In [1]:
import os
from dataset_utils import *

In [2]:
train_path = os.path.join(IMDB_DATA_PATH, "train")
test_path  = os.path.join(IMDB_DATA_PATH, "test")

#Main training set - Large Movie Review Dataset (IMDB)
((train_texts, train_labels), (test_texts, test_labels)) = load_dataset(train_path=train_path, test_path=test_path)

#Review Polarity Datasets - used as additional test data
(v1_texts, v1_labels), (v2_texts, v2_labels) = load_polarity(v1_path=POLARITY_v1_DATA_PATH, v2_path=POLARITY_v2_DATA_PATH)

In [3]:
import numpy as np

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_extraction.text import TfidfVectorizer

MAX_FEATURES = 5000

#encoding largest dataset, training vectorizer and selector --> to be used for encoding other datasets
def get_ngram_dataset(train_texts, train_labels, test_texts, test_labels, min_df=5):
    vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=min_df) 
    
    X_train = vectorizer.fit_transform(train_texts)  
    selector = SelectKBest(score_func=f_classif, k=min(MAX_FEATURES, X_train.shape[1]))    

    y_train = np.array(train_labels)
    y_test = np.array(test_labels)
    
    X_train = selector.fit_transform(X_train, y_train).toarray()

    X_test = vectorizer.transform(test_texts)
    X_test = selector.transform(X_test).toarray()
    
    return ((X_train, y_train), (X_test, y_test), (vectorizer, selector))

#get the encoded data as well as the trained vectorizer and selector 
(X_train, y_train), (X_test, y_test), (vectorizer, selector) = get_ngram_dataset(
    train_texts, train_labels, test_texts, test_labels
)

In [4]:
def vectorize(texts, labels, vectorizer, selector):
    X = vectorizer.transform(texts)
    y = np.array(labels)

    X = selector.transform(X).toarray()
    
    return (X, y)

In [5]:
X_v1_test, y_v1_test = vectorize(v1_texts, v1_labels, vectorizer=vectorizer, selector=selector)
X_v2_test, y_v2_test = vectorize(v2_texts, v2_labels, vectorizer=vectorizer, selector=selector)

In [6]:
#placeholder benchmark naive bayes model
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#X_sm_train, X_sm_test, y_sm_train, y_sm_test = train_test_split(X_sm, y_sm, test_size=0.33, random_state=42)

clf = ComplementNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(accuracy_score(clf.predict(X_v1_test), y_v1_test))
print(accuracy_score(clf.predict(X_v2_test), y_v2_test))

0.87912
0.7323203901706997
0.825


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

#For now just wrapping already processed vectors
class NgramMovieDataset(Dataset):
    def __init__(self, X, y, device):        
        self.x = torch.from_numpy(X).to(device, dtype=torch.float32)
        self.y = torch.from_numpy(y).to(device, dtype=torch.float32)
            
    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])
    
    def __len__(self):
        return self.x.shape[0]


device = ("cuda" if torch.cuda.is_available() else "cpu")

def get_dataloader(X, y, device, batch_size=64, shuffle=True):
    dataset = NgramMovieDataset(X=X, y=y, device=device)
    return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)

train_dataloader = get_dataloader(X=X_train, y=y_train, device=device)
test_dataloader  = get_dataloader(X=X_test, y=y_test, device=device)

v1_test_dataloader = get_dataloader(X=X_v1_test, y=y_v1_test, device=device)
v2_test_dataloader = get_dataloader(X=X_v2_test, y=y_v2_test, device=device)

In [8]:
from torch import nn

input_dim = X_train.shape[1]

class SentimentCLF(nn.Module):
    def __init__(self, n_units):
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(input_dim, n_units),
            nn.ReLU(),
            nn.Linear(n_units, n_units),
            nn.ReLU(),
            nn.Linear(n_units, 1)
        )
            
        self.dropout = nn.Dropout(p=0.2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        dropped = self.dropout(x)
        logits = self.linear_stack(dropped)
        
        return self.sigmoid(logits)  
    

def train(dataloader: DataLoader, model: nn.Module, loss_fn, optimizer):
    
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader): 
        model.train()
        
        pred = model(X).squeeze()    
        loss = loss_fn(pred, y)
         
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
                   
                     
def test(dataloader: DataLoader, model: nn.Module, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X).squeeze()
            test_loss += loss_fn(pred, y).item()
            
            out_class = (pred > 0.5).float()      
            correct += (out_class==y).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
model = SentimentCLF(n_units=512).to(device)
loss_fn   = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)

epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn)
print("Done!")

for dataloader in [test_dataloader, v1_test_dataloader, v2_test_dataloader]:
    test(dataloader=dataloader, model=model, loss_fn=loss_fn)

Epoch 1
-------------------------------
loss: 0.691947  [   64/25000]
loss: 0.223031  [ 6464/25000]
loss: 0.445481  [12864/25000]
loss: 0.264050  [19264/25000]
Epoch 2
-------------------------------
loss: 0.225481  [   64/25000]
loss: 0.220166  [ 6464/25000]
loss: 0.129743  [12864/25000]
loss: 0.179622  [19264/25000]
Epoch 3
-------------------------------
loss: 0.169082  [   64/25000]
loss: 0.236171  [ 6464/25000]


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from skorch import NeuralNetClassifier

net = NeuralNetClassifier(
    module=SentimentCLF,
    module__n_units=512,
    lr = 0.001,
    criterion=nn.BCELoss,   
    device='cuda',
    max_epochs=15,
    optimizer=torch.optim.Adam,
    batch_size=64,
)

In [ ]:
X_train_sk = torch.from_numpy(X_train).to(dtype=torch.float32)
y_train_sk = torch.from_numpy(y_train).to(dtype=torch.float32).unsqueeze(1)

In [ ]:
net.initialize()
net.module_

In [ ]:
net.fit(X_train_sk, y_train_sk)

In [ ]:
X_test_sk = torch.from_numpy(X_test).to(device=device, dtype=torch.float32)
y_test_sk = torch.from_numpy(y_test).to(device='cpu', dtype=torch.float32)

pred = net.predict(X_test_sk)
accuracy_score(pred, y_test_sk)

In [ ]:
from sklearn.model_selection import GridSearchCV


# deactivate skorch-internal train-valid split and verbose logging
net.set_params(train_split=False, verbose=0)
param_grid = {
    'net__lr': [0.001, 0.01],
    'net__max_epochs': [10, 15, 20],
    'net__module__n_units': [100, 500, 1000],
    'vectorizer__min_df': [2, 3, 5, 7],
}
gs = GridSearchCV(pipe, param_grid=param_grid, refit=False, cv=3, scoring='accuracy', verbose=2)

gs.fit(train_texts, train_labels)
#print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))

In [ ]:
net.set_params(lr=0.001, max_epochs=10, module__n_units=100)
net.initialize()
net.fit(X_train_sk, y_train_sk)